# Tutorial 1: Your First Adaptive Agent

**Goal**: Build an agent that automatically adapts when tools fail

**Time**: 5 minutes

**What you'll learn**:
- Install LRS-Agents
- Create a basic adaptive agent
- See automatic adaptation in action

---

## The Problem

Standard agents loop forever when tools fail:

```python
# Standard ReAct agent
agent.run("Fetch data from API")
# API fails → agent retries same action → timeout
```

LRS agents **detect failure and automatically try alternatives**.

## Step 1: Installation

```bash
pip install lrs-agents
pip install langchain-anthropic  # For LLM
```

In [ ]:
# Verify installation
import lrs
print(f"LRS version: {lrs.__version__}")

## Step 2: Create Tools

We'll create two tools:
- `APITool`: Fast but unreliable (fails 50% of the time)
- `CacheTool`: Slower but reliable fallback

In [ ]:
from lrs.core.lens import ToolLens, ExecutionResult
import random

class APITool(ToolLens):
    """Unreliable API - fails 50% of the time"""
    
    def __init__(self):
        super().__init__(
            name="api_fetch",
            input_schema={'type': 'object', 'required': ['query']},
            output_schema={'type': 'string'}
        )
    
    def get(self, state: dict) -> ExecutionResult:
        """Simulate unreliable API call"""
        self.call_count += 1
        
        if random.random() < 0.5:  # 50% failure rate
            self.failure_count += 1
            print(f"  ❌ API failed (call #{self.call_count})")
            return ExecutionResult(
                success=False,
                value=None,
                error="API timeout",
                prediction_error=0.9  # High surprise!
            )
        else:
            print(f"  ✓ API succeeded (call #{self.call_count})")
            return ExecutionResult(
                success=True,
                value="API data: [1, 2, 3]",
                error=None,
                prediction_error=0.1
            )
    
    def set(self, state: dict, observation: str) -> dict:
        return {**state, 'data': observation}


class CacheTool(ToolLens):
    """Reliable cache - always works"""
    
    def __init__(self):
        super().__init__(
            name="cache_fetch",
            input_schema={'type': 'object', 'required': ['query']},
            output_schema={'type': 'string'}
        )
    
    def get(self, state: dict) -> ExecutionResult:
        """Always succeeds (but slower)"""
        self.call_count += 1
        print(f"  ✓ Cache hit (call #{self.call_count})")
        return ExecutionResult(
            success=True,
            value="Cached data: [1, 2, 3]",
            error=None,
            prediction_error=0.0
        )
    
    def set(self, state: dict, observation: str) -> dict:
        return {**state, 'data': observation}

## Step 3: Create LRS Agent

**Key difference from standard agents**: We register `CacheTool` as a fallback for `APITool`

In [ ]:
from lrs import create_lrs_agent
from lrs.core.registry import ToolRegistry
from unittest.mock import Mock

# Create tool registry
registry = ToolRegistry()

api_tool = APITool()
cache_tool = CacheTool()

# Register with fallback chain
registry.register(api_tool, alternatives=["cache_fetch"])
registry.register(cache_tool)

# Create mock LLM (for this tutorial, we'll use simple policy generation)
mock_llm = Mock()

# Build LRS graph
from lrs.integration.langgraph import LRSGraphBuilder

builder = LRSGraphBuilder(
    llm=mock_llm,
    registry=registry,
    preferences={
        'data_retrieved': 5.0,  # High reward for getting data
        'error': -3.0            # Penalty for errors
    }
)

agent = builder.build()

## Step 4: Run the Agent

Watch what happens when the API fails:

In [ ]:
# Initialize state
initial_state = {
    "messages": [{"role": "user", "content": "Fetch user data"}],
    "belief_state": {"goal": "Fetch data"},
    "precision": {},
    "prediction_errors": {},
    "current_policy": [],
    "candidate_policies": [],
    "G_values": {},
    "tool_history": [],
    "adaptation_count": 0,
    "current_hbn_level": "abstract"
}

# Run agent
print("🤖 Starting LRS Agent...\n")
result = agent.invoke(initial_state)

print("\n" + "="*50)
print("RESULTS")
print("="*50)
print(f"Total adaptations: {result['adaptation_count']}")
print(f"Tools used: {len(result['tool_history'])}")
print(f"Final precision: {result['precision']}")

print("\nExecution trace:")
for i, entry in enumerate(result['tool_history'], 1):
    status = "✓" if entry['success'] else "✗"
    print(f"  {i}. {status} {entry['tool']} (error: {entry['prediction_error']:.2f})")

## What Just Happened?

If the API failed, you saw:

```
1. ✗ api_fetch (error: 0.90)     ← High prediction error!
   → Precision drops
   → Agent replans
2. ✓ cache_fetch (error: 0.00)   ← Automatic fallback
```

**The agent didn't retry the same failed action.** It:
1. Detected high prediction error
2. Precision collapsed (confidence dropped)
3. Automatically explored the cache alternative

**This is adaptation, not just error handling.**

## Key Concepts

### Prediction Error (ε)
- **What it is**: `|expected - observed|`
- **When it's high**: Tool behaved unexpectedly
- **What it triggers**: Precision update

### Precision (γ)
- **What it is**: Agent's confidence in its world model
- **Range**: 0 (no confidence) to 1 (total confidence)
- **Effect**: Controls exploration vs exploitation

### The Adaptation Loop
```
Execute → Observe → Calculate Error → Update Precision → Replan
```

## Try It Yourself

Experiment with different failure rates:

In [ ]:
# Make API very unreliable (90% failure)
class UnreliableAPI(ToolLens):
    def get(self, state):
        self.call_count += 1
        if random.random() < 0.9:  # 90% failure!
            self.failure_count += 1
            return ExecutionResult(False, None, "Failed", 0.95)
        return ExecutionResult(True, "Data", None, 0.05)
    
    def set(self, state, obs):
        return state

# Question: How quickly does the agent switch to cache?
# Try it and see!

## Next Steps

- **Tutorial 2**: Understand how precision actually works (Beta distributions)
- **Tutorial 3**: Build complex tool chains with composition
- **Tutorial 4**: Run the full Chaos Scriptorium benchmark

---

**Congratulations!** You've built your first adaptive agent. 🎉